<h1>Meme trends analysis</h1>
This project aims to analyze trends in memes in order to visualize and correlate data.

In [6]:
import pandas as pd
import plotly as pl
from utils import download_dataset

memes = pd.read_csv(download_dataset.get_path())

Dataset URL: https://www.kaggle.com/datasets/podsyp/a-lot-of-memes-info-stats